In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bdtp4.parser import parse
dataset = {e['id']: e for e in parse('../data/amazon-meta.txt') if e}

In [2]:
import lzma
import json
with lzma.open('../data/dataset.json.xz') as file:
    dataset = json.load(file)

In [3]:
import random
random.choice(list(dataset.values()))

{'id': 256623,
 'ASIN': '0764508210',
 'title': 'Microsoft FrontPage 2002 for Dummies (With CD-ROM)',
 'group': 'Book',
 'salesrank': '36517',
 'similar': ['0735613001',
  '0072133643',
  '076453582X',
  '0201741431',
  '0735612846'],
 'categories': [['Books', '283155'],
  ['Subjects', '1000'],
  ['Computers & Internet', '5'],
  ['Web Development', '3510'],
  ['Internet Commerce', '3554'],
  ['Web Site Design', '3565'],
  ['Books', '283155'],
  ['Subjects', '1000'],
  ['Computers & Internet', '5'],
  ['Networking', '3652'],
  ['Internet, Groupware, & Telecommunications', '3705'],
  ['Internet Publishing', '3710'],
  ['Books', '283155'],
  ['Subjects', '1000'],
  ['Computers & Internet', '5'],
  ['Software', '4053'],
  ['Introductory Guides', '549660'],
  ['For Dummies: General', '4123'],
  ['Books', '283155'],
  ['Subjects', '1000'],
  ['Computers & Internet', '5'],
  ['Graphics & Illustration', '4134'],
  ['Desktop Publishing', '4094'],
  ['General', '171130'],
  ['Books', '283155'],


In [7]:
import psycopg2 as pg

conn = pg.connect(dbname='bdtp4', user='mard', password='1234')

In [8]:
cursor = conn.cursor()

In [40]:
mogrifiedValues = []
for value in dataset.values():
    _id = value['id']
    _asin = value['ASIN']
    _title = value.get('title', None)
    _group = value.get('group', None)
    _salesRank = value.get('salesrank', None)
    
    mv = cursor.mogrify('(%s,%s,%s,%s,%s)', (_id, _asin, _title, _group, _salesRank))
    mogrifiedValues.append(mv)

In [41]:
values = b','.join(mogrifiedValues)

In [42]:
cursor.execute(b'INSERT INTO product VALUES ' + values)
conn.commit()

In [38]:
cursor.execute('DELETE FROM product')

In [43]:
categories = 